In [5]:
import scipy
import numpy as np
import time

def gaussian(x, parameter):
  heavy_computation = np.random.uniform(0,0.01)
  time.sleep(heavy_computation)
  return np.exp(-(x* parameter)**2)

1. What is the difference between multiprocessing and multithreading?

In [ ]:
# Multithreading is a technique where multiple threads are spawned by a process to do different tasks at about the same time,
# just one after the other. This gives you the illusion that the threads are running in parallel,but they are actually run in a concurrent manner.
# In Python, the Global Interpreter Lock (GIL) prevents the threads from running simultaneously.
# The recent version of python - 3.12 has introduced concurrent.features which allows multithreading.
# Python 3.12 will also allow real multithreading/concurrency by enabling multiple sub-interpreters within the same system.
import concurrent.futures
with concurrent.futures.ProcessPoolExecutor() as executor:
  parameter = np.random.randint(1,100)
  proc = executor.submit(scipy.integrate.quad, gaussian, -np.inf, np.inf, args=(parameter))
  print(proc.result())

# Multiprocessing is a technique where parallelism in its truest form is achieved.
# Multiple processes are run across multiple CPU cores, which do not share the resources among them.
# Each process can have many threads running in its own memory space.
# In Python, each process has its own instance of Python interpreter doing the job of executing the instructions.




(0.018272720112427997, 4.291770625904841e-11)


2. Write a simple program of multiprocessing.

In [ ]:
import multiprocessing
print("Number of cpu : ", multiprocessing.cpu_count())

def sleep():
    print('Starting to sleep')
    print('Done sleeping')

start = time.time()
p1 =  multiprocessing.Process(target= sleep)
p2 =  multiprocessing.Process(target= sleep)
p1.start()
p2.start()

print(f"\nDone in {time.time()-start} seconds")
# The time log print statement gets printed along with the function. This is because along with the multi-process instances triggered for the sleep function,
#  the main code of the function got executed separately in parallel.

Number of cpu :  2
Starting to sleep
Starting to sleep

Done in 0.033699750900268555 seconds
Done sleeping
Done sleeping


3. How does a particular python code utilize shared memory using the multiprocessing
module to create a process that modifies shared data and what values are printed for
the shared variables “number” and “array” after the process is executed?

In [1]:
import multiprocessing
shared_memory = multiprocessing.Array('f', range(5))

def integrate(gaussian, lower_limit, upper_limit, i, shared_memory):
  start_time = time.time()
  integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(np.random.randint(1,100)))
  shared_memory[i] = integral
  print('The value of ', i, 'th integral is', integral)
  print(i, ' took ', time.time()-start_time, 'sec ...')

In [6]:
import time
start_time = time.time()
for i in range(5):
  integrate(gaussian, -np.inf, np.inf, i, shared_memory)
print(time.time()-start_time)
shared_memory[:]
# Multiple processes can modify the shared value safely using shared memory.
# It is protected such that only one process can access the value or array at a time

The value of  0 th integral is 0.018855892030909743
0  took  2.4378445148468018 sec ...
The value of  1 th integral is 0.05371072275471262
1  took  1.3632612228393555 sec ...
The value of  2 th integral is 0.08862269254527581
2  took  1.3114612102508545 sec ...
The value of  3 th integral is 0.1042619912297362
3  took  1.0505335330963135 sec ...
The value of  4 th integral is 0.02769459142039869
4  took  1.7577672004699707 sec ...
7.928936004638672


[0.01885589212179184,
 0.05371072143316269,
 0.08862268924713135,
 0.10426199436187744,
 0.027694592252373695]

4. How does the Python code utilize the “multiprocessing.Pool” to execute the square
function in parallel for the provided list of numbers, and what is the output of the
result variable printed at the end?

In [9]:
input = []
def integrate_map(agruments):
  gaussian, lower_limit, upper_limit, i, parameter = agruments
  start_time = time.time()
  integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(parameter))
  #print('The value of ', i, 'th integral is', integral)
  print(i, 'took', time.time()-start_time, 'secs ...\n')

  return [i, integral]
for i in range(5):
    parameter = np.random.randint(1,100)
    input.append([gaussian,-np.inf, np.inf, i, parameter])

start_time = time.time()

with multiprocessing.Pool() as pool:
  for integral in pool.map(integrate_map, input):
    print('integrate value', integral)
print('That took ', time.time()-start_time, 'sec ...')

0 took 1.1534502506256104 secs ...

1 took 1.778228759765625 secs ...

2 took 1.6566166877746582 secs ...

3 took 1.892298936843872 secs ...

4 took 1.2898812294006348 secs ...

integrate value [0, 0.09328704478450081]
integrate value [1, 0.029056620506647802]
integrate value [2, 0.03692612189386491]
integrate value [3, 0.02110064108220852]
integrate value [4, 0.055389182840797385]
That took  4.2501609325408936 sec ...


5. Explain how the Python code utilizes a shared value and a lock from the
multiprocessing module to ensure synchronized access to the shared_value.
Additionally, what is the final value of the shared_value printed after all processes
have completed their tasks?


In [1]:
import multiprocessing

# Define a function that will be executed by the child process
def modify_shared_data(shared_value, increment):
    with shared_value.get_lock():
        shared_value.value += increment

# Create a shared integer value
shared_value = multiprocessing.Value('i', 0)

# Create two processes that will increment the shared value
process1 = multiprocessing.Process(target=modify_shared_data, args=(shared_value, 1))
process2 = multiprocessing.Process(target=modify_shared_data, args=(shared_value, 2))

# Start the processes
process1.start()
process2.start()

# Wait for the processes to finish
process1.join()
process2.join()

# The shared value has been modified by both processes
print("Final Shared Value:", shared_value.value)


Final Shared Value: 3


6. Write down a sample code which computes the difference between the speed of
numpy and jax numpy ?


In [ ]:
import jax
def f(x):
  return -4*x*x*x + 9*x*x + 6*x - 3

x = np.arange(0,1000000)
y = jax.numpy.arange(0,1000000)
%timeit f(x)
%timeit jax.jit(f)(y)

7.14 ms ± 683 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
221 µs ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


7. Explain how the code utilizes JAX NumPy to define and apply a custom function
custom_function to the input array x, and what is the result of the custom function for
the provided values in the array x? where,
x = jnp.array([0.0, jnp.pi / 2, jnp.pi])
and return the following operation :
jnp.sin(x) + jnp.cos(x)
Explain the code briefly.


In [ ]:
import jax.numpy as jnp
import jax
import numpy as np

def custom_function(x):
  return jnp.cos(x)+jnp.sin(x)
x=jnp.array([0.0,jnp.pi/2,jnp.pi])
%timeit custom_function(x)

def numpy_function(x):
  return np.cos(x)+np.sin(x)
y=np.array([0.0,np.pi/2,np.pi])
%timeit numpy_function(y)

20 µs ± 2.73 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.01 µs ± 519 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


8. There is an array: [1.0, -1.0, 0.0, and 1.5]. Compute this array for all values in such a
way if the value is greater than 0 then it will return sin(x) or else will display cos(x)

In [ ]:
x=np.array([1.0,-1.0,0.0,1.5])
np.where(x>0,np.sin(x),np.cos(x))

array([0.84147098, 0.54030231, 1.        , 0.99749499])

9. Explain the utilization of “multiprocessing.Pipe” for inter-process communication,
where one process (sender) sends a series of messages through the pipe, and another
process (receiver) retrieves and prints these messages. How the communication is
managed between the parent and child processes, and how are the messages
exchanged and processed using the pipe? Explain with a suitable code.

In [14]:
def integrate_pipe(gaussian, lower_limit, upper_limit, i, send_end, parameter):
  # a=time.time()
  integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(parameter))
  send_end.send([i, integral])
  # print(time.time()-a,"\n")
  #print('The value of ', i, 'th integral is', integral)

rec_end, send_end = multiprocessing.Pipe()
start_time = time.time()
for i in range(5):
  parameter = np.random.randint(1,100)
  process = multiprocessing.Process(target=integrate_pipe, args=(gaussian,-np.inf, np.inf, i, send_end, parameter))
  process.start()
  integral = rec_end.recv()
  print('value of integral is', integral, '\n')
print('That took ', time.time()-start_time, 'sec ...')

value of integral is [0, 0.026855361377356307] 

value of integral is [1, 0.018657408956900164] 

value of integral is [2, 0.020852398245947246] 

value of integral is [3, 0.04544753463860298] 

value of integral is [4, 0.057175930674371496] 

That took  9.197922468185425 sec ...


10. Create one numpy array (a, b) and one jax_numpy array (c, d) of 100X100 size.
Perform dot(.) product between the numpy array (a, b), and jax_numpy array (c, d).
Compute the time between these two arrays

In [ ]:
import numpy as np
a=np.random.randn(100,100)
b=np.random.randn(100,100)

from jax import random
key = random.PRNGKey(758493)
c=random.uniform(key, shape=(100,100))
key = random.PRNGKey(758494)
d=random.uniform(key, shape=(100,100))

import jax
%timeit np.dot(a,b)
%timeit jax.numpy.dot(c,d)

104 µs ± 29.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
48.5 µs ± 787 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
